In [1]:
from tkinter import *
from PIL import Image
from tensorflow.keras.models import load_model
import numpy
import PIL.ImageOps  
import cv2 as cv

In [ ]:
#load model
model = load_model('model.h5')

canvas_width = 200
canvas_height = 300

def paint( event ):
   color = "black"
   x1, y1 = ( event.x - 1 ), ( event.y - 1 )
   x2, y2 = ( event.x + 1 ), ( event.y + 1 )
   w.create_oval( x1, y1, x2, y2, fill = color, outline = color, width = 10)
    

def save_image():
    w.postscript(file = 'fileName' + '.eps')
    img = Image.open('fileName' + '.eps')
    img.save('fileName' + '.png', 'png')
    img = Image.open('fileName.png')
    inverted_image = PIL.ImageOps.invert(img)
    inverted_image.save('new_name.png')
    im = cv.imread('new_name.png')
    imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
    ret, th = cv.threshold(imgray, 127, 255, 0)
    contours = cv.findContours(th, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[0]
    
    x, y, v, h = cv.boundingRect(contours)
    cv.rectangle(im, (x, y), (x + v, y + h), (255, 0, 0), 1)
    top = int(0.05 * th.shape[0])
    bottom = top
    left = int( 0.05 * th.shape[1])
    right = left
    th_up = cv.copyMakeBorder(th, top, bottom, left, right, cv.BORDER_REPLICATE)
    roi = th[y-top: y+h+bottom, x-left: x+v+right]
    img = cv.resize(roi, (28, 28), interpolation = cv.INTER_AREA)
    img = img.reshape(1, 28, 28, 1)/255

    if max(model.predict(img)[0]) > 0.95:
        number = numpy.argmax(model.predict(img))
        cnv_prdn.create_text( 100, 150, text = number, font = ('Times', '50', 'bold italic'))
    else:
        number = 'Not a number'
        cnv_prdn.create_text( 100, 150, text = number, font = ('Times', '15', 'bold italic'))
    

def clear():
    w.delete('all')
    cnv_prdn.delete('all')
    
master = Tk()
master.title( "Digit Prediction" )
w = Canvas(master, bg= 'white',
           width=canvas_width, 
           height=canvas_height)
w.pack(expand = YES, fill = BOTH, side = 'left')
w.bind( "<B1-Motion>", paint )
cnv_prdn = Canvas(master, bg= 'white',
           width=canvas_width, 
           height=canvas_height)
cnv_prdn.pack(expand = YES, fill = BOTH, side = 'right')
lbl_prdn = Label(cnv_prdn, text = 'Prediction', width=25, font = ('Helvetica', '10', 'bold italic'))
lbl_prdn.pack(side = 'top')

btn_submit = Button( master, text = "Predict",  width = 5, command = save_image )
btn_submit.pack( side = 'right', padx = (5, 50))
btn_clear = Button(master, text = 'Clear',  width = 5, command = clear)
btn_clear.pack(side = 'right', padx = (50, 5))
    
mainloop()


0.8415632
0.99432087
0.99999964
0.98981327
